In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
from copy import deepcopy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Attributes:
- sound
- prev
- next
- len
- status
- f1
- f1_edge_delta (delta between stable and transition edge points)
- f1_inner_delta (delta between stable or transition inner edge points)
- f2
- f2_edge_delta
- f2_inner_delta
- intensity
- len (by segment)
- pitch
- time
- stable f1 coef
- stable f1 bias
- trans f1 coef
- trans f1 bias
- stable f2 coef
- stable f2 bias
- trans f2 coef
- trans f2 bias

In [2]:
plt.style.use('ggplot')

In [3]:
PATH = r'C:\Users\PC\Desktop\stu\phon17_18\praat'
NA = '--undefined--'
CONS = [("к", "п'"), ("г", "л'"), ("к", "й"), ("к", "т'"), ("г", "п'"), ("к", "й"),
        ("г", "т'"), ("г", "й"), ("к", "л'"), ("к", "п'"), ("к", "т'"), ("г", "л'")]
HEADER = ['sound', 'sound_idx', 'segment', 'segment_idx', 'time', 'pitch', 'intensity', 'f1', 'f2']

In [4]:
COLUMNS = [
           'rec', 'status', 'sound', 'prev', 'next', 'len', 'ratio', 'st_len', 'tr_len',
           'f1_inner_delta', 'f1_edge_delta', 'f2_inner_delta', 'f2_edge_delta',
           'f1_st_coef', 'f1_st_bias', 'f1_tr_coef', 'f1_tr_bias',
           'f2_st_coef', 'f2_st_bias', 'f2_tr_coef', 'f2_tr_bias',
           'pitch_st_1', 'pitch_st_2', 'pitch_st_3', 'pitch_st_4', 'pitch_st_5',
           'pitch_tr_1', 'pitch_tr_2', 'pitch_tr_3', 'pitch_tr_4', 'pitch_tr_5',
           'intensity_st_1', 'intensity_st_2', 'intensity_st_3', 'intensity_st_4', 'intensity_st_5',
           'intensity_tr_1', 'intensity_tr_2', 'intensity_tr_3', 'intensity_tr_4', 'intensity_tr_5',
           'f1_st_1', 'f1_st_2', 'f1_st_3', 'f1_st_4', 'f1_st_5',
           'f1_tr_1', 'f1_tr_2', 'f1_tr_3', 'f1_tr_4', 'f1_tr_5',
           'f2_st_1', 'f2_st_2', 'f2_st_3', 'f2_st_4', 'f2_st_5',
           'f2_tr_1', 'f2_tr_2', 'f2_tr_3', 'f2_tr_4', 'f2_tr_5'
           ]

In [5]:
def read_csv(sound_name, build=True):
    def get_sound_idx(line):
        return int(line.split(',')[1])
    
    cont = list()
    _idx = -1
    cnt = 0
    cV = None
    f = open('%s\%s.csv' % (PATH, sound_name), 'r', encoding='utf-8').readlines()[1:]
     
    for i, line in enumerate(f):
        c_idx = get_sound_idx(line)
        if c_idx != _idx and line.split(',')[0].strip():
            if cV is not None:
                cont.append(cV)
            cV = V(sound_name, c_idx, cnt)
            _idx = c_idx
            cnt += 1
        if line.split(',')[0].strip():
            cV.add_line(line)
    
    # the last vowel
    cont.append(cV)
    
    for v in cont: v.build()
    
    return cont

In [23]:
#vs = read_csv('')

In [30]:
class V:
    
    def __init__(self, rec, sound_idx, idx):
        """
        rec: номер записи
        sound_idx: индекс в текстгриде
        idx: порядковый номер гласного в тексте
        """
        self.rec = rec
        self.sound_idx = sound_idx
        self.idx = idx
         
        self.lines = list()
        self.info_line = 'sound=%s, prev=%s, next=%s, len=%s, status=%s'
        self.atrs = ['time', 'pitch', 'intensity', 'f1', 'f2']
        self.atr_idxs = [4, 5, 6, 7, 8]
        self.st_coef = dict()
        self.tr_coef = dict()
        self.sound_map = {'e': 'е',
                          'a': 'а',
                          'u': 'у'
                         }
        
    def __str__(self):
        return 'V(%s, %s)' % (self.rec, self.sound_idx)
            
    __repr__ = __str__
    
    def info(self):
        return self.info_line % (self.sound, self.prev, self.next,
                                 round(self.len, 2), self.status)
    
    def add_line(self, x):
        self.lines.append(x.strip())
        
    def to_int(self, line, idx):
        if NA in line.split(',')[idx]:
            return np.nan
        return int(line.split(',')[idx])
    
    def to_float(self, line, idx):
        if NA in line.split(',')[idx]:
            return np.nan
        return float(line.split(',')[idx])
    
    def get_status(self):
        """
        0: ok
        1: uncertain transition
        2: uncertain spectrum
        """
        x = self.lines[0].split(',')[0].strip() 
        if x[0] == '?':
            self.status = 1
        elif x[0] == '!':
            self.status = 2
        else:
            self.status = 0
    
    def get_sound(self):
        x = self.lines[0].split(',')[0].strip()
        x = x.replace('?', '').replace('!', '')
        self.sound = self.sound_map[x]
        
    def get_prev(self):
        self.prev = CONS[self.idx][0]
        
    def get_next(self):
        self.next = CONS[self.idx][1]
        
    def get_len(self):
        _len = self.to_float(self.lines[-1], 4) - self.to_float(self.lines[0], 4) 
        self.len = _len * 1000
        
    def delta_prop(self, x):
        props = [0]

        for i in range(1, len(x)):
            props.append(abs(((100 * x[i]) / x[i-1]) - 100))

        return np.array(props)
    
    def build_st_atr(self):
        self.st_atr = {a: np.array([]) for a in self.atrs}
        
        for i in range(len(self.lines)):
            if self.lines[i].split(',')[2].strip() == 'st':
                for j, atr in enumerate(self.atrs):
                    #self.tr_atr[atr].append(self.to_float(self.lines[i], self.atr_idxs[j]))
                    self.st_atr[atr] = np.append(self.st_atr[atr], [self.to_float(self.lines[i], self.atr_idxs[j])])
                    
        if self.st_atr['time'].any():
            self.st_atr['len'] = self.st_atr['time'][-1] - self.st_atr['time'][0]
            self.st_atr['len'] *= 1000
        else:
            self.st_atr['len'] = None
            
        if self.st_atr['f1'].any():
            self.st_atr['f1_inner_delta'] = self.st_atr['f1'][-1] - self.st_atr['f1'][0]
            self.st_atr['f1_delta_prop'] = self.delta_prop(self.st_atr['f1'])
        else:
            self.st_atr['f1_inner_delta'] = None
            self.st_atr['f1_delta_prop'] = None
            
        if self.st_atr['f2'].any():
            self.st_atr['f2_inner_delta'] = self.st_atr['f2'][-1] - self.st_atr['f2'][0]
            self.st_atr['f2_delta_prop'] = self.delta_prop(self.st_atr['f2'])
        else:
            self.st_atr['f2_inner_delta'] = None
            self.st_atr['f2_delta_prop'] = None
            
        self.st_atr['f1_edge_delta'] = None
        self.st_atr['f2_edge_delta'] = None
        
        self.st_atr['smoothed_f1'] = False
        self.st_atr['smoothed_f2'] = False
            
    def build_tr_atr(self):
        self.tr_atr = {a: np.array([]) for a in self.atrs}
        
        for i in range(len(self.lines)):
            if self.lines[i].split(',')[2].strip() == 'tr':
                for j, atr in enumerate(self.atrs):
                    #self.tr_atr[atr].append(self.to_float(self.lines[i], self.atr_idxs[j]))
                    self.tr_atr[atr] = np.append(self.tr_atr[atr], [self.to_float(self.lines[i], self.atr_idxs[j])])
        
        if self.tr_atr['time'].any():
            self.tr_atr['len'] = self.tr_atr['time'][-1] - self.tr_atr['time'][0]
            self.tr_atr['len'] *= 1000
        else:
            self.tr_atr['len'] = None
        
        if self.tr_atr['f1'].any():
            self.tr_atr['f1_inner_delta'] = self.tr_atr['f1'][-1] - self.tr_atr['f1'][0]
            self.tr_atr['f1_delta_prop'] = self.delta_prop(self.tr_atr['f1'])
        else:
            self.tr_atr['f1_inner_delta'] = None
            self.tr_atr['f1_delta_prop'] = None
        
        if self.tr_atr['f2'].any():
            self.tr_atr['f2_inner_delta'] = self.tr_atr['f2'][-1] - self.tr_atr['f2'][0]
            self.tr_atr['f2_delta_prop'] = self.delta_prop(self.tr_atr['f2'])
        else:
            self.tr_atr['f2_inner_delta'] = None
            self.tr_atr['f2_delta_prop'] = None
        
        if self.st_atr['f1'].any() and self.tr_atr['f1'].any():
            self.tr_atr['f1_edge_delta'] = self.tr_atr['f1'][-1] - self.st_atr['f1'][-1]
        else:
            self.tr_atr['f1_edge_delta'] = None
            
        if self.st_atr['f2'].any() and self.tr_atr['f2'].any():
            self.tr_atr['f2_edge_delta'] = self.tr_atr['f2'][-1] - self.st_atr['f2'][-1]
        else:
            self.tr_atr['f2_edge_delta'] = None
            
        self.tr_atr['smoothed_f1'] = False
        self.tr_atr['smoothed_f2'] = False
            
    def get_ratio(self):
        if self.tr_atr['len'] is not None:
            self.ratio = (100 * self.tr_atr['len']) / self.len
        else:
            self.ratio = 0.0
        
    def mean(self, x, by):
        if by == 'st':
            return np.mean(self.st_atr[x])
        else:
            return np.mean(self.tr_atr[x])
        
    def std(self, x, by):
        if by == 'st':
            return np.std(self.st_atr[x])
        else:
            return np.std(self.tr_atr[x])
        
    def regress(self, X, y, normalize=False):
        """
        data: `st_atr` or `tr_atr`
        target: `f1` or `f2`
        """
        if not X.any():
            return None, None
        
        lr = LinearRegression(normalize=normalize)
        lr.fit(X, y)
        return lr.coef_[0], lr.intercept_
        
    def fit(self, target, by):
        """
        self.`st|tr`_coef = {target: [fitted coef_, fitted intercept_]}
        """
        if by == 'st':
            coef, intercept = self.regress(self.st_atr['time'].reshape(-1, 1), self.st_atr[target])
            self.st_coef[target] = [coef, intercept]
        else:
            coef, intercept = self.regress(self.tr_atr['time'].reshape(-1, 1), self.tr_atr[target])
            self.tr_coef[target] = [coef, intercept]
            
    def rmse(self, target, by):
        if by == 'st':
            score = mean_squared_error(self.st_atr[target],
                                       self.st_atr['time']  * self.st_coef[target][0] + self.st_coef[target][1])
            return round(np.sqrt(score), 4)
        else:
            score = mean_squared_error(self.tr_atr[target],
                                       self.tr_atr['time']  * self.tr_coef[target][0] + self.tr_coef[target][1])
            return round(np.sqrt(score), 4)
        
    def r2(self, target, by):
        if by == 'st':
            score = r2_score(self.st_atr[target],
                                       self.st_atr['time']  * self.st_coef[target][0] + self.st_coef[target][1])
            return round(score, 4)
        else:
            score = r2_score(self.tr_atr[target],
                                       self.tr_atr['time']  * self.tr_coef[target][0] + self.tr_coef[target][1])
            return round(score, 4)
            
    def plot_fit(self, target, by, target_style='scatter', ylabel=None, legend=True, fsize=(6, 4)):
        plt.figure(figsize=fsize)
        if by == 'st':
            if target_style == 'scatter':
                plt.scatter(self.st_atr['time'], self.st_atr[target], color='b', alpha=0.5)
                plt.plot(self.st_atr['time'], self.st_atr[target], color='b', alpha=0.45)
            else:
                plt.plot(self.st_atr['time'], self.st_atr[target], color='b', alpha=0.9)
            coef = self.st_coef[target][0]
            intercept = self.st_coef[target][1]
            plt.plot(self.st_atr['time'], self.st_atr['time'] * coef + intercept,
                     color='r', alpha=0.6)
        else:
            if target_style == 'scatter':
                plt.scatter(self.tr_atr['time'], self.tr_atr[target], color='b', alpha=0.5)
                plt.plot(self.tr_atr['time'], self.tr_atr[target], color='b', alpha=0.45)
            else:
                plt.plot(self.tr_atr['time'], self.tr_atr[target], color='b', alpha=0.9)
            coef = self.tr_coef[target][0]
            intercept = self.tr_coef[target][1]
            plt.plot(self.tr_atr['time'], self.tr_atr['time'] * coef + intercept,
                     color='r', alpha=0.6)
            
        if legend:
            _coef = 'coef = %s' % round(coef[0], 2)
            _intercept = 'intercept = %s' % round(intercept, 2)
            rmse = 'RMSE = %s' % self.rmse(target, by)
            r2 = 'R2 = %s' % self.r2(target, by)
            els = [Line2D([0], [0], color='w', label=_coef),
                   Line2D([0], [0], color='w', label=_intercept),
                   Line2D([0], [0], color='w', label=rmse),
                   Line2D([0], [0], color='w', label=r2)]
            plt.legend(handles=els, bbox_to_anchor=(1, 1))
        
        plt.title('%s' % self.info())
        plt.xlabel('Time, ms.')
        if ylabel is None:
            plt.ylabel(target)
        else:
            plt.ylabel(ylabel)
    
    def build(self):
        self.get_sound()
        self.get_status()
        self.get_prev()
        self.get_next()
        self.get_len()
        self.build_st_atr()
        self.build_tr_atr()
        self.get_ratio()
        self.fit('f1', 'st')
        self.fit('f2', 'st')
        self.fit('f1', 'tr')
        self.fit('f2', 'tr')
        

In [81]:
vs = read_csv('5')

In [44]:
#for v in vs: print(v.len)

In [82]:
_v = deepcopy(vs[0])
_v.build()

In [50]:
_v.st_atr

{'f1': array([], dtype=float64),
 'f1_edge_delta': None,
 'f1_inner_delta': None,
 'f2': array([], dtype=float64),
 'f2_edge_delta': None,
 'f2_inner_delta': None,
 'intensity': array([], dtype=float64),
 'len': None,
 'pitch': array([], dtype=float64),
 'time': array([], dtype=float64)}

In [53]:
a = np.array([1])
np.append(a, np.array([None]))
a

array([1])

In [7]:
COLUMNS = [
           'rec', 'status', 'sound', 'prev', 'next', 'len', 'ratio', 'st_len', 'tr_len',
           'f1_inner_delta_st', 'f1_edge_delta_st', 'f2_inner_delta_st', 'f2_edge_delta_st',
           'f1_inner_delta_tr', 'f1_edge_delta_tr', 'f2_inner_delta_tr', 'f2_edge_delta_tr',
           'f1_st_coef', 'f1_st_bias', 'f1_tr_coef', 'f1_tr_bias',
           'f2_st_coef', 'f2_st_bias', 'f2_tr_coef', 'f2_tr_bias',
           'pitch_st_1', 'pitch_st_2', 'pitch_st_3', 'pitch_st_4', 'pitch_st_5',
           'pitch_tr_1', 'pitch_tr_2', 'pitch_tr_3', 'pitch_tr_4', 'pitch_tr_5',
           'intensity_st_1', 'intensity_st_2', 'intensity_st_3', 'intensity_st_4', 'intensity_st_5',
           'intensity_tr_1', 'intensity_tr_2', 'intensity_tr_3', 'intensity_tr_4', 'intensity_tr_5',
           'f1_st_1', 'f1_st_2', 'f1_st_3', 'f1_st_4', 'f1_st_5',
           'f1_tr_1', 'f1_tr_2', 'f1_tr_3', 'f1_tr_4', 'f1_tr_5',
           'f2_st_1', 'f2_st_2', 'f2_st_3', 'f2_st_4', 'f2_st_5',
           'f2_tr_1', 'f2_tr_2', 'f2_tr_3', 'f2_tr_4', 'f2_tr_5'
           ]

# : - key-val marker;  # - iterable marker

PARAM_MAP = [
             'rec', 'status', 'sound', 'prev', 'next', 'len', 'ratio', 'st_atr:len', 'tr_atr:len',
             'st_atr:f1_inner_delta', 'st_atr:f1_edge_delta', 'st_atr:f2_inner_delta', 'st_atr:f2_edge_delta',
             'tr_atr:f1_inner_delta', 'tr_atr:f1_edge_delta', 'tr_atr:f2_inner_delta', 'tr_atr:f2_edge_delta',
             '#st_coef:f1', '#tr_coef:f1', '#st_coef:f2', '#tr_coef:f2',
             '#st_atr:pitch', '#tr_atr:pitch',
             '#st_atr:intensity', '#tr_atr:intensity',
             '#st_atr:f1', '#tr_atr:f1', '#st_atr:f2', '#tr_atr:f2'
            ]

In [59]:
#_v.__dict__

In [8]:
def build_df(vs):
    def spread(x):
        if not x.any():
            return np.zeros
    recs = list()
    
    for v in vs:
        _rec = list()
        for prm in PARAM_MAP:
            if ':' in prm:
                key = prm.split(':')[0]
                val = prm.split(':')[1]
                if '#' in prm:
                    key = key.replace('#', '')
                    x = v.__dict__[key][val]
                    if list(x):
                        _rec.extend(list(x))
                    else:
                        _rec.extend([None for _ in range(5)])
                else:
                    x = v.__dict__[key][val]
                    _rec.append(x)
            else:
                x = v.__dict__[prm]
                _rec.append(x)
            # print('prm: %s\nrec: %s\nPARAM: %s\n' % (prm, _rec, COLUMNS[len(_rec)-1]))

        assert len(_rec) == len(COLUMNS)

        recs.append(_rec)
        
    df = pd.DataFrame.from_records(recs, columns=COLUMNS)
    df.replace([None], [np.nan], inplace=True)
    
    return df

In [9]:
def load(recs):
    E = list()
    A = list()
    U = list()
    
    for rec in recs:
        vs = read_csv(str(rec))
        for v in vs:
            if v.sound == 'е':
                E.append(v)
            elif v.sound == 'а':
                A.append(v)
            else:
                U.append(v)
                
    assert len(E) == len(A) == len(U)
    
    return E, A, U

In [41]:
E, A, U = load([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [42]:
df = build_df([*E, *A, *U])

In [120]:
df.groupby(['sound', 'next'])['f2_tr_1', 'f2_tr_2', 'f2_tr_3', 'f2_tr_4', 'f2_tr_5'].mean().mean(axis=1)

sound  next
а      й       1693.905855
       л'      1620.647773
       п'      1600.923938
       т'      1674.012291
е      й       2191.346321
       л'      2016.738175
       п'      2159.236918
       т'      2292.709067
у      й       1189.920833
       л'      1160.285075
       п'       985.556275
       т'      1251.134389
dtype: float64

In [15]:
df[df.status == 0].sound.value_counts()

а    33
у    21
е    19
Name: sound, dtype: int64

In [21]:
def delta_prop(x):
    props = [0]
    
    for i in range(1, len(x)):
        props.append(abs(((100 * x[i]) / x[i-1]) - 100))
        
    return np.array(props)

In [22]:
delta_prop([4, 5, 4])

array([ 0., 25., 20.])

In [26]:
def group_by(V, global_atr=[], segment_atr=[], prev=None, _next=None):
    for i, v in enumerate(V):
        if v.status > 0:
            continue
            
        if prev is not None:
            if v.prev != prev:
                continue
        
        if _next is not None:
            if v.next != _next:
                continue
        
        print('%s.  --- [%s%s%s], rec: %s ---' % (i + 1, v.prev, v.sound, v.next, v.rec))
        if global_atr:
            print('GLOBAL')
            for atr in global_atr:
                print('\t%s: %s' % (atr, v.__dict__[atr]))
        if segment_atr:
            print('SEGMENT')
            for atr in segment_atr:
                print('\tstable:\t%s: %s' % (atr, v.st_atr[atr]))
                print('\ttrans.:\t%s: %s' % (atr, v.tr_atr[atr]))
        print('\n')

In [27]:
_nexts = ["п'", "т'", "л'", "й"]

In [45]:
group_by(U, ['status', 'len'], ['len', 'f2', 'f2_delta_prop', 'smoothed_f2', 'f2_inner_delta', 'f2_edge_delta'], _next=_nexts[1])

12.  --- [гут'], rec: 3 ---
GLOBAL
	status: 0
	len: 181.9760784386233
SEGMENT
	stable:	len: 113.9908704489585
	trans.:	len: 67.98520798966479
	stable:	f2: [621.72256064 654.44480067 673.18888204 670.94906774 717.88178303]
	trans.:	f2: [ 717.88178303  807.81575361  988.40238181 1221.15815381 2006.84958914]
	stable:	f2_delta_prop: [ 0.         11.04413897  2.86411953  0.33271707  6.99497436]
	trans.:	f2_delta_prop: [ 0.         12.5276853  22.35492777 23.54868587 64.33985908]
	stable:	smoothed_f2: True
	trans.:	smoothed_f2: False
	stable:	f2_inner_delta: -17.814317351406544
	trans.:	f2_inner_delta: 1288.9678061105296
	stable:	f2_edge_delta: None
	trans.:	f2_edge_delta: 1288.9678061105296


16.  --- [гут'], rec: 4 ---
GLOBAL
	status: 0
	len: 168.2461586568138
SEGMENT
	stable:	len: 99.95506467049609
	trans.:	len: 68.29109398631772
	stable:	f2: [781.64753144 822.7868752  787.48198227 805.11302434 783.5840012 ]
	trans.:	f2: [ 783.5840012   913.9904355  1143.79332377 1445.3156522  1924.637697

In [40]:
def smoothing(V, mode='both', atrs=['f2'], props={'st': 5, 'tr': 10}):
    def left_smooth(idxs):
        """
        idxs[0]: initial point
        idxs[1]: second point
        """
        for atr in atrs:
            if v.st_atr[atr].any() and v.st_atr[atr][idxs[0]] > v.st_atr[atr][idxs[1]]:
                v.st_atr[atr][idxs[0]] = v.st_atr[atr][idxs[1]] - ((v.st_atr[atr][idxs[1]] * props['st']) / 100)
                v.st_atr['smoothed_%s' % atr] = True

            if v.tr_atr[atr].any() and v.tr_atr[atr][idxs[0]] > v.tr_atr[atr][idxs[1]]:
                v.tr_atr[atr][idxs[0]] = v.tr_atr[atr][idxs[1]] - ((v.tr_atr[atr][idxs[1]] * props['tr']) / 100)
                v.tr_atr['smoothed_%s' % atr] = True
    
    def right_smooth(idxs):
        """
        idxs[0]: penultimate point
        idxs[1]: ultimate point
        """
        for atr in atrs:
            if v.st_atr[atr].any() and v.st_atr[atr][idxs[1]] < v.st_atr[atr][idxs[0]]:
                v.st_atr[atr][idxs[1]] = v.st_atr[atr][idxs[0]] + ((v.st_atr[atr][idxs[0]] * props['st']) / 100)
                v.st_atr['smoothed_%s' % atr] = True

            if v.tr_atr[atr].any() and v.tr_atr[atr][idxs[1]] < v.tr_atr[atr][idxs[0]]:
                v.tr_atr[atr][idxs[1]] = v.tr_atr[atr][idxs[0]] + ((v.tr_atr[atr][idxs[0]] * props['tr']) / 100)
                v.tr_atr['smoothed_%s' % atr] = True
        
        
    for v in V:
        # ДОБАВИТЬ ПРОВЕРКУ НА МОНОТОННО НИСХОДЯЩИЙ /е/
        if mode == 'left':
            left_smooth([0, 1])
                
        elif mode == 'right':
            right_smooth([-2, -1])
        
        else:
            left_smooth([0, 1])
            right_smooth([-2, -1])
    
    return V

In [43]:
U = smoothing(U, 'left')

In [13]:
def print_out(recs):
    for rec in recs:
        vs = read_csv(str(rec))
        print('----- REC. %s -----\n' % rec)
        for v in vs:
            if v.status == 0:
                status = 'OK'
            else:
                status = 'PROBLEM'
                
            print('vowel: /%s/\nstatus: %s\ncontext: [%s %s %s]\nlen: %s\nstable len: %s\ntrans. len: %s\nstable F2: %s\ntrans. F2: %s\n'
                     % (v.sound, status, v.prev, v.sound, v.next, v.len,
                        v.st_atr['len'], v.tr_atr['len'], v.st_atr['f2'], v.tr_atr['f2'])
                     )

In [14]:
print_out([3, 4, 5, 6])

----- REC. 3 -----

vowel: /а/
status: PROBLEM
context: [к а п']
len: 150.01633946943116
stable len: 96.83649245713255
trans. len: 53.17984701229861
stable F2: [2383.63637706 1443.6624476  1117.098973   2450.80003704 2499.33503661]
trans. F2: [2499.33503661 1973.57184936 1956.46123814 1416.42373777 1285.58042764]

vowel: /у/
status: PROBLEM
context: [г у л']
len: 166.5438783899944
stable len: 64.62304483223136
trans. len: 101.92083355776305
stable F2: [ 674.22811213  642.46813073  567.71883805  575.01718356 2451.5330662 ]
trans. F2: [2451.5330662  2423.68061997 2449.00500999 2418.41928549 2387.546493  ]

vowel: /у/
status: OK
context: [∅ у й]
len: 137.46754278684392
stable len: 59.12102995517188
trans. len: 78.34651283167204
stable F2: [624.48098667 572.97054368 592.28741296 596.57422465 621.24441868]
trans. F2: [ 621.24441868  693.92162556  759.35893719  929.1481382  1701.31434043]

vowel: /е/
status: OK
context: [к е т']
len: 135.67661251248353
stable len: 135.67661251248353
trans. l

In [16]:
print('vowel: /%s/\nstatus: %s\ncontext: [%s %s %s]\nlen: %s\nstable len: %s\ntrans. len: %s\nstable F2: %s\ntrans. F2: %s'
     % (_v.sound, _v.status, _v.prev, _v.sound, _v.next, _v.len,
        _v.st_atr['len'], _v.tr_atr['len'], _v.st_atr['f2'], _v.tr_atr['f2'])
     )

vowel: /а/
status: 0
context: [г а л']
len: 178.0015946406195
stable len: 86.07722428166653
trans. len: 91.92437035895296
stable F2: [1412.60365932 1404.640879   1320.09121147 1321.13331052 1294.20035577]
trans. F2: [1294.20035577 1319.14740989 1348.51455683 1411.16975418 1468.18198081]


In [42]:
vs = read_csv('5')

KeyError: ''

In [40]:
vs

[V(6, 2),
 V(6, 4),
 V(6, 6),
 V(6, 8),
 V(6, 10),
 V(6, 12),
 V(6, 14),
 V(6, 16),
 V(6, 18),
 V(6, 20),
 V(6, 22)]

In [26]:
vs[0].lines

['?a, 2, st, 2, 21.098546051358344, --undefined--, 74.04784699796359, 780.7128247251602, 1785.0800489204034',
 '?a, 2, st, 2, 21.117913349849772, 126.84509272753431, 80.02664418359653, 738.7000905771761, 2323.69514271682',
 '?a, 2, st, 2, 21.1372806483412, 126.40436492825674, 83.07601331748558, 875.301108234363, 2525.307299462071',
 '?a, 2, st, 2, 21.156647946832628, 124.45821241072044, 84.25461695165832, 984.7231511375725, 2477.8641679074067',
 '?a, 2, st, 2, 21.195382543815477, 118.7396996792083, 85.01303914172488, 943.3685804258288, 2494.2865969511986',
 '?a, 2, tr, 3, 21.195382543815477, 118.7396996792083, 85.01303914172488, 943.3685804258288, 2494.2865969511986',
 '?a, 2, tr, 3, 21.206018513217938, 117.32776043831485, 84.67347150860823, 970.772707410967, 2494.820286977796',
 '?a, 2, tr, 3, 21.2166544826204, 115.77673199494662, 83.85291902163311, 962.4761556766317, 2388.5580006502737',
 '?a, 2, tr, 3, 21.22729045202286, 114.11919947205664, 82.28201107237678, 1126.9356457738588, 234